Sraavika Vaddadi 
<br> 
11/6/24

# **Analysis of Geopolitical Events on Oil Prices**
## Part 2 - Data Cleaning and Pre-Processing

In [3]:
import pandas as pd
import numpy as np
import requests
import json

## Import GPR and EIA data files

In [5]:
# Import GPR index data from CSV into a dataframe 

gpr_index = pd.read_csv('data_gpr_export.csv', encoding = 'utf-8')
print(gpr_index.shape)

(1499, 115)


In [6]:
# Import crude price data from CSV into a dataframe 

price_data = pd.read_csv('price_data_raw.csv', encoding = 'utf-8')
print(price_data.shape)

(918, 11)


## GPR Data Cleaning

In [8]:
# Include only necessary columns 

gpr_index = gpr_index[['month', 'GPR', 'GPRT', 'GPRA', 'N10', 'SHARE_GPR', 'SHAREH_CAT_1', 'SHAREH_CAT_2', 'SHAREH_CAT_3', 'SHAREH_CAT_4', 'SHAREH_CAT_5', 'SHAREH_CAT_6', 'SHAREH_CAT_7', 'SHAREH_CAT_8']]

In [9]:
# rename columns 

gpr_index = gpr_index.rename(columns={
    'month': 'period',
    'GPRT': 'gpr_threats',
    'GPRA': 'gpr_acts',
    'SHARE_GPR': 'gpr_article_proportion',
    'N10': 'total_articles',
    'SHAREH_CAT_1': 'cat_1',
    'SHAREH_CAT_2': 'cat_2',
    'SHAREH_CAT_3': 'cat_3',
    'SHAREH_CAT_4': 'cat_4',
    'SHAREH_CAT_5': 'cat_5',
    'SHAREH_CAT_6': 'cat_6',
    'SHAREH_CAT_7': 'cat_7', 
    'SHAREH_CAT_8': 'cat_8'})

In [10]:
# identify column datatypes

print(gpr_index.dtypes)

period                     object
GPR                       float64
gpr_threats               float64
gpr_acts                  float64
total_articles            float64
gpr_article_proportion    float64
cat_1                     float64
cat_2                     float64
cat_3                     float64
cat_4                     float64
cat_5                     float64
cat_6                     float64
cat_7                     float64
cat_8                     float64
dtype: object


In [11]:
# change month column to date time format 

gpr_index['period'] = pd.to_datetime(gpr_index['period'])

In [12]:
# filter to include records only from May 1986

gpr_index = gpr_index[gpr_index['period'] >= '1987-05-01']

In [13]:
# convert GPR to a logarithmic scale 

gpr_index['log_GPR'] = np.log(gpr_index['GPR'])

In [14]:
# normalize gpr data to make it on a scale of 0 to 1

gpr_index['normalized_gpr'] = (gpr_index['log_GPR'] - gpr_index['log_GPR'].min()) / (gpr_index['log_GPR'].max() - gpr_index['log_GPR'].min())

In [15]:
# view dataframe after cleaning

display(gpr_index.head())
print(gpr_index.shape)

,period,GPR,gpr_threats,gpr_acts,total_articles,gpr_article_proportion,cat_1,cat_2,cat_3,cat_4,cat_5,cat_6,cat_7,cat_8,log_GPR,normalized_gpr
1048,1987-05-01,98.65,97.37,95.24,17405.0,2.96,0.36,0.07,0.61,0.75,0.20,0.49,0.48,0.52,4.591578,0.359965
1049,1987-06-01,94.18,99.83,81.83,16921.0,2.82,0.40,0.02,0.94,0.57,0.10,0.34,0.43,0.44,4.545208,0.341954
1050,1987-07-01,98.87,107.53,88.26,16894.0,2.97,0.24,0.04,0.99,0.64,0.26,0.35,0.47,0.65,4.593806,0.360830
1051,1987-08-01,99.97,115.48,87.60,16642.0,3.00,0.46,0.09,1.02,0.60,0.25,0.57,0.47,0.46,4.604870,0.365128
1052,1987-09-01,121.17,143.45,97.54,17335.0,3.63,0.63,0.09,1.18,1.01,0.22,0.65,0.61,0.42,4.797195,0.439831


(451, 16)


## EIA Data Cleaning

In [17]:
# include only necessary columns

price_data = price_data[['period', 'product-name', 'value']]

In [18]:
# rename columns 

price_data = price_data.rename(columns = {'product-name': 'product', 'value': 'price'})

In [19]:
# identify column datatypes

print(price_data.dtypes)

period      object
product     object
price      float64
dtype: object


In [20]:
# change period column to date time format 

price_data['period'] = pd.to_datetime(price_data['period'])

In [21]:
# filter to include records only from May 1987

price_data = price_data[price_data['period'] >= '1987-05-01']

In [22]:
# convert prices to a logarithmic scale

price_data['log_price'] = np.log(price_data['price'])

In [23]:
# normalize log price data to make it on a scale of 0 to 1

price_data['normalized_price'] = (price_data['log_price'] - price_data['log_price'].min()) / (price_data['log_price'].max() - price_data['log_price'].min())

In [24]:
# view dataframe after cleaning

display(price_data)
print(price_data.shape)

,period,product,price,log_price,normalized_price
0,2024-11-01,WTI Crude Oil,69.95,4.247781,0.751519
1,2024-11-01,UK Brent Crude Oil,74.35,4.308784,0.774869
2,2024-10-01,WTI Crude Oil,71.99,4.276527,0.762522
3,2024-10-01,UK Brent Crude Oil,75.63,4.325853,0.781403
4,2024-09-01,UK Brent Crude Oil,74.02,4.304335,0.773166
...,...,...,...,...,...
897,1987-07-01,UK Brent Crude Oil,19.86,2.988708,0.269581
898,1987-06-01,WTI Crude Oil,20.07,2.999226,0.273607
899,1987-06-01,UK Brent Crude Oil,18.86,2.937043,0.249805
900,1987-05-01,UK Brent Crude Oil,18.58,2.922086,0.244080


(902, 5)


In [25]:
# save cleaned wti and brent data to a new csv 

price_data.to_csv('price_data.csv', header = True, index = False, encoding = 'utf-8')

## Merge Pricing and GPR Data 

In [27]:
# merge gpr and wti price data 

gpr_price = pd.merge(gpr_index, price_data, on='period', how='inner')
display(gpr_price.head())
print(gpr_price.shape)

,period,GPR,gpr_threats,gpr_acts,total_articles,gpr_article_proportion,cat_1,cat_2,cat_3,cat_4,cat_5,cat_6,cat_7,cat_8,log_GPR,normalized_gpr,product,price,log_price,normalized_price
0,1987-05-01,98.65,97.37,95.24,17405.0,2.96,0.36,0.07,0.61,0.75,0.20,0.49,0.48,0.52,4.591578,0.359965,UK Brent Crude Oil,18.58,2.922086,0.244080
1,1987-05-01,98.65,97.37,95.24,17405.0,2.96,0.36,0.07,0.61,0.75,0.20,0.49,0.48,0.52,4.591578,0.359965,WTI Crude Oil,19.44,2.967333,0.261399
2,1987-06-01,94.18,99.83,81.83,16921.0,2.82,0.40,0.02,0.94,0.57,0.10,0.34,0.43,0.44,4.545208,0.341954,WTI Crude Oil,20.07,2.999226,0.273607
3,1987-06-01,94.18,99.83,81.83,16921.0,2.82,0.40,0.02,0.94,0.57,0.10,0.34,0.43,0.44,4.545208,0.341954,UK Brent Crude Oil,18.86,2.937043,0.249805
4,1987-07-01,98.87,107.53,88.26,16894.0,2.97,0.24,0.04,0.99,0.64,0.26,0.35,0.47,0.65,4.593806,0.360830,WTI Crude Oil,21.34,3.060583,0.297093


(902, 20)


In [28]:
# save the cleaned file into a new csv 

gpr_price.to_csv('gpr_price.csv', header = True, index = False, encoding = 'utf-8')
